In [1]:
import cv2
import sys, os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import numpy as np
from numpy import asarray

# DeepFake Detection Algorithm
Input : $D_{real}$, $D_{fake}$ were centered by subtracting the mean of the real training data.


## Create $D_{real}$

In [2]:
# Read images
img_real = []
path = '/content/drive/MyDrive/American_University/2021_Fall/DATA-793-001_Data Science Practicum/Datasets/manipulated_sequences/Deepfakes/raw/videos/data_test'

for root, _, files in os.walk(path):
    current_directory_path = os.path.abspath(root)
    for f in files:
        name, ext = os.path.splitext(f)
        if ext == ".jpg":
            current_image_path = os.path.join(current_directory_path, f)
            current_image = imageio.imread(current_image_path)
            img_real.append(current_image)
img_real = np.array(img_real, dtype=object)
for img in img_real:
  print(img.shape)
print(len(img_real))

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(480, 854, 3)
(480, 854, 3)
10


## Global Centering for each image

In [ ]:
def meanSubtraction(arr):
  new_arr = []
  for i in range(len(arr)):
    img = arr[i]
    img = img.astype(np.float32) # convert from integers to floats
    mean = img.mean() # calculate global mean
    img = img - mean # centering of pixels
    img /= img.std()
    new_arr.append(img)
  new_arr = np.array(new_arr, dtype=object)
  return new_arr

img_real = meanSubtraction(img_real)
print(len(img_real))
print(img_real[0])

# Preprocessing and data tensor organization

In [ ]:
print(img_real.shape)
[U_real, S_real, V_real] = np.linalg.svd(img_real[0])

(10,)


In [ ]:
for i in range(len(img_real)):
  [U_real, S_real, V_real] = np.linalg.svd(img_real[i])
  

(720, 3, 3)


# Testing Zone

In [ ]:
"""
img_df = []           
for img in images:
  img = img.flatten()
  img_df.append(img)
  print(img.shape)
img_df = np.array(img_df, dtype=object)
print(img_df)
"""